<a href="https://colab.research.google.com/github/MaxSivo/Housing-analysis-using-XGBoost-and-RandomSearchCV/blob/main/housing_with_xgboost_and_randomcv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


**Read data and identify null records**

In [ ]:
df_train = pd.read_csv('../input/home-data-for-ml-course/train.csv', index_col=0)
df_test = pd.read_csv('../input/home-data-for-ml-course/test.csv', index_col=0)

In [ ]:
df_train.shape

(1460, 80)

In [ ]:
df_test.shape

(1459, 79)

In [ ]:
train_null_cols = []
for col in df_train.columns:
    if df_train[col].isnull().sum() > 0:
        train_null_cols.append(col)

df_test[train_null_cols].isnull().sum()

LotFrontage      227
Alley           1352
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinType2      42
Electrical         0
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
dtype: int64

In [ ]:
test_null_cols = []
for col in df_test.columns:
    if df_test[col].isnull().sum() > 0:
        test_null_cols.append(col)

df_test[test_null_cols].isnull().sum()

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64

**Remove cols with significant number of NaN values**

In [ ]:
bad_cols = ['PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu', 'Alley']
df_train.drop(axis=1, inplace=True, columns=bad_cols)
df_test.drop(axis=1, inplace=True, columns=bad_cols)
df_train.shape

(1460, 75)

**Seperate categorical and numerical columns**

In [ ]:
num_cols = []
for col in df_train.columns:
    if df_train[col].dtypes != 'object':
        num_cols.append(col)

num_cols

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SalePrice']

In [ ]:
cat_cols = []
for col in df_train.columns:
    if df_train[col].dtypes == 'object':
        cat_cols.append(col)
cat_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

**Fill Nan values with the mean and mode for numerical and categorical columns respectfully**

In [ ]:
for col in num_cols:
    df_train[col] = df_train[col].fillna(df_train[col].mean())

for col in cat_cols:
    df_train[col] = df_train[col].fillna(df_train[col].mode()[0])
df_train.isna().any().sum()

0

In [ ]:
for col in num_cols[:-1]:
    df_test[col] = df_test[col].fillna(df_test[col].mean())

for col in cat_cols[:-1]:
    df_test[col] = df_test[col].fillna(df_test[col].mode()[0])
df_test.isna().any().sum()

0

**One hot encoding**

In [ ]:
df_train_ohe = pd.get_dummies(df_train)
df_test_ohe = pd.get_dummies(df_test)
df_train_ohe.shape

(1460, 271)

In [ ]:
df_test_ohe.shape

(1459, 254)

**Align test df to the train df since train df has more variables**

In [ ]:
df_train_ohe, df_test_ohe = df_train_ohe.align(df_test_ohe, join='left', axis=1)
df_test_ohe.drop(axis=1, inplace=True, columns='SalePrice')

df_test_ohe.shape

(1459, 270)

In [ ]:
df_train_ohe.shape

(1460, 271)

**Use XGBoost**


In [ ]:
from xgboost import XGBRegressor

classifier = XGBRegressor()

**Splitting the data**

In [ ]:
X = df_train_ohe.drop(axis=1, columns='SalePrice')
y = df_train_ohe.SalePrice

df_final = df_test_ohe.copy()

****

**Hyperparameter optimization**

In [ ]:
params = {
    'n_estimators': [100, 250, 500, 800, 900, 1000, 1200],
    'max_depth': [3, 4, 5, 6, 8, 10],
    'learning_rate': [0.05, 0.08, 0.1, 0.15],
    'min_child_depth': [1, 2, 3, 4]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_cv = RandomizedSearchCV(classifier,
            param_distributions = params,
            cv = 5, 
            n_jobs = -1,
            n_iter = 50,
            scoring = 'neg_mean_absolute_error',
            verbose = 3
            )
            

In [ ]:
random_cv.fit(X, y)

random_cv.best_estimator_

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[08:31:57] WARNING: ../src/learner.cc:627: 
Parameters: { "min_child_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[CV 2/5] END learning_rate=0.05, max_depth=4, min_child_depth=3, n_estimators=100;, score=-17017.598 total time=   2.3s
[08:31:59] WARNING: ../src/learner.cc:627: 
Parameters: { "min_child_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[CV 5/5] END learning_rate=0.05, max_depth=4, min_child_depth=3, n_estimators=100;, sc

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, ...)

In [ ]:
random_cv.best_params_

{'n_estimators': 1000,
 'min_child_depth': 3,
 'max_depth': 3,
 'learning_rate': 0.05}

In [ ]:
final_classifier = XGBRegressor(n_estimators = 1000, min_child_depth = 3, max_depth = 3, learning_rate = 0.05, random_state=0)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = -1 * cross_val_score(final_classifier, X, y,
                                  cv=5,
                                  scoring='neg_mean_absolute_error')

[08:53:07] WARNING: ../src/learner.cc:627: 
Parameters: { "min_child_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[08:53:15] WARNING: ../src/learner.cc:627: 
Parameters: { "min_child_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[08:53:23] WARNING: ../src/learner.cc:627: 
Parameters: { "min_child_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually 

In [ ]:
scores.mean()

15333.491239030394

In [ ]:
final_classifier.fit(X, y)

In [ ]:
final_pred = final_classifier.predict(df_final)

In [ ]:
output = pd.DataFrame({'Id': df_final.index,
                       'SalePrice': final_pred})
output.to_csv('submission.csv', index=False)